In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 1. generalized online tensor factorization (GOTF)

### 1.1 data preparation

In [2]:
from GOCPT import datasets, simulation, otf, gotf
import numpy as np

# load Columbia FACE 3D tensor
X = datasets.FACE_3D()

# simulate one mode growth (typically the last mode)
# use 30% as preparation and increase 3 slices at each step
[X_0, X_inc_ls] = simulation.growth_1d(X, prep=0.3, inc=1)


        ---------- new OTF setting ------------
        base tensor size: (112, 92, 120),
        new tensor increment size: (112, 92, 1),
        tensor will be updated 279 times.
        


### 1.2 our GOCPT and GOCPTE model

In [3]:
# initialize model
# model = gotf.GOCPTE(X_0, R=5)
model = gotf.GOCPT(X_0, R=5)

# update model with new slices, new_R can change during for-loop
for increments in X_inc_ls[:10]:
    new_R = np.random.choice([5,6,7,8])
    model.update(increments, new_R=new_R)

Initial PoF Metric: 0.7467675346834539
This model does not need to prepare aux!

1-th update Success! PoF: 0.7468, run_time: 0.08425s
2-th update Success! PoF: 0.7473, run_time: 0.1072s
3-th update Success! PoF: 0.7477, run_time: 0.04188s
4-th update Success! PoF: 0.7484, run_time: 0.1095s
5-th update Success! PoF: 0.7488, run_time: 0.04334s
6-th update Success! PoF: 0.7488, run_time: 0.04211s
7-th update Success! PoF: 0.7488, run_time: 0.0428s
8-th update Success! PoF: 0.749, run_time: 0.04388s
9-th update Success! PoF: 0.7489, run_time: 0.07901s
10-th update Success! PoF: 0.7484, run_time: 0.04441s


### 1.3 baseline models

In [4]:
# baseline models for online tensor factorization (OTF)

baseline = otf.MAST(X_0, R=5)
# baseline = otf.OnlineCPD(X_0, R=5)
# baseline = otf.SDT(X_0, R=5)
# baseline = otf.RLST(X_0, R=5)
# baseline = otf.CPStream(X_0, R=5)

# update model with new slices
for increments in X_inc_ls[:10]:
    baseline.update(increments)

Initial PoF Metric: 0.7472945275495273
This model does not need to prepare aux!

1-th update, PoF: 0.7472, run_time: 0.309s
2-th update, PoF: 0.7475, run_time: 0.1346s
3-th update, PoF: 0.7477, run_time: 0.1367s
4-th update, PoF: 0.7482, run_time: 0.2308s
5-th update, PoF: 0.7486, run_time: 0.2247s
6-th update, PoF: 0.7484, run_time: 0.2241s
7-th update, PoF: 0.7483, run_time: 0.2294s
8-th update, PoF: 0.7485, run_time: 0.163s
9-th update, PoF: 0.7483, run_time: 0.2337s
10-th update, PoF: 0.7479, run_time: 0.2929s


# 2. generalized online tensor completion (GOTC)

### 2.1 data preparation

In [5]:
from GOCPT import datasets, simulation, otc, gotc
import numpy as np

# load synthetic data
[masked_X, mask] = datasets.syn_data(R=5, size=(5, 10, 15, 50), dist='unif', sparsity=0.95)

# simulate one mode growth (typically the last mode)
# use 30% as preparation and increase 3 slices at each step
[[X_0, mask_0], [X_inc_ls, mask_inc_ls]] = simulation.growth_1d([masked_X, mask], prep=0.3, inc=3)


        ---------- new OTC setting ------------
        base tensor size: (5, 10, 15, 15),
        new tensor increment size: (5, 10, 15, 3),
        tensor will be updated 12 times.
        


### 2.2 Our GOCPT and GOCPTE model

In [8]:
# model initialization
# model = gotc.GOCPTE([X_0, mask_0], R=5)
model = gotc.GOCPT([X_0, mask_0], R=5)

# update model with new slices, new_R can change during for-loop
for increments in zip(X_inc_ls, mask_inc_ls):
    # simulate value update， missing filling，change of rank
    new_value_update = simulation.value_update(model.X, model.mask, percent=10, amp=0.05)
    new_missing_fill = simulation.missing_fill(model.X, model.mask, percent=10, factors=model.factors)
    new_R = np.random.choice([5,6,7,8])
    model.update(increments, new_R=new_R, value_update=new_value_update, miss_fill=new_missing_fill)

Initial PoF Metric: 0.8647733153804781
This model does not need to prepare aux!

number of newly updated entries: 10 / 575
number of newly filled entries: 10 / 10675
1-th update Success! PoF: 0.8496, run_time: 0.02193s

number of newly updated entries: 10 / 695
number of newly filled entries: 10 / 12805
2-th update Success! PoF: 0.8572, run_time: 0.02532s

number of newly updated entries: 10 / 819
number of newly filled entries: 10 / 14931
3-th update Success! PoF: 0.8569, run_time: 0.03279s

number of newly updated entries: 10 / 929
number of newly filled entries: 10 / 17071
4-th update Success! PoF: 0.8446, run_time: 0.03274s

number of newly updated entries: 10 / 1069
number of newly filled entries: 10 / 19181
5-th update Success! PoF: 0.8491, run_time: 0.03527s

number of newly updated entries: 10 / 1195
number of newly filled entries: 10 / 21305
6-th update Success! PoF: 0.8555, run_time: 0.0689s

number of newly updated entries: 10 / 1332
number of newly filled entries: 10 / 2341

### 2.3 Our GOCPT and GOCPTE model

In [9]:
model = otc.OnlineSGD([X_0, mask_0], R=5)

for increments in zip(X_inc_ls, mask_inc_ls):
    model.update(increments)

Initial PoF Metric: 0.8901285386728529
This model does not need to prepare aux!

1-th update, PoF: 0.8548426208435884, run_time: 0.01519465446472168s
2-th update, PoF: 0.843109684101022, run_time: 0.018860816955566406s
3-th update, PoF: 0.8399727843184965, run_time: 0.016878366470336914s
4-th update, PoF: 0.829827985531642, run_time: 0.016872406005859375s
5-th update, PoF: 0.825048417687894, run_time: 0.016789913177490234s
6-th update, PoF: 0.8108696969276589, run_time: 0.016805171966552734s
7-th update, PoF: 0.804390444553433, run_time: 0.016820192337036133s
8-th update, PoF: 0.8045874667339361, run_time: 0.015585899353027344s
9-th update, PoF: 0.8093925328909948, run_time: 0.01680469512939453s
10-th update, PoF: 0.8106501870133198, run_time: 0.014002799987792969s
11-th update, PoF: 0.80776765934681, run_time: 0.015578985214233398s
12-th update, PoF: 0.80791040455736, run_time: 0.013554096221923828s
